In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

1. tabular
2. spatial
3. time series
4. combined
5. image

In [ ]:
# Load your tabular dataset
data = pd.read_csv('your_tabular_data.csv')
X = data.drop('target_column', axis=1).values  # Features
y = data['target_column'].values  # Target variable

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

# Define the feedforward neural network
class FeedforwardNN(nn.Module):
    def __init__(self, input_size):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer for regression

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = FeedforwardNN(input_size=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
# Load your spatial dataset (e.g., images)
data = np.load('your_spatial_data.npy')  # Assuming data is in .npy format
X = data['images']  # Shape: (num_samples, channels, height, width)
y = data['masks']   # Shape: (num_samples, num_classes, height, width)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Define the U-Net architecture
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder1 = self.conv_block(1, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.decoder1 = self.upconv_block(256, 128)
        self.decoder2 = self.upconv_block(128, 64)
        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            self.conv_block(out_channels, out_channels)
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)
        enc3 = self.encoder3(enc2)
        dec1 = self.decoder1(enc3)
        dec2 = self.decoder2(dec1 + enc2)  # Skip connection
        return self.final_conv(dec2 + enc1)  # Skip connection

# Initialize model, loss function, and optimizer
model = UNet()
criterion = nn.BCEWithLogitsLoss()  # For binary segmentation
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Load your time series dataset
data = pd.read_csv('your_time_series_data.csv')
X = data.drop('target_column', axis=1).values  # Features
y = data['target_column'].values  # Target variable

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors and reshape for RNN
X_train_tensor = torch.FloatTensor(X_train).view(-1, 1, X_train.shape[1])  # (batch_size, seq_len, input_size)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test).view(-1, 1, X_test.shape[1])
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)
# Define the LSTM architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # Output layer for regression

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Get the last time step's output
        return out

# Initialize model, loss function, and optimizer
input_size = X_train.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
model = LSTMModel(input_size, hidden_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
# Load your mixed dataset
data = pd.read_csv('your_mixed_data.csv')
X_tabular = data.drop('target_column', axis=1).values  # Tabular features
X_time_series = np.load('your_time_series_data.npy')['time_series']  # Time series data
y = data['target_column'].values  # Target variable

# Split the dataset
X_tabular_train, X_tabular_test, X_time_series_train, X_time_series_test, y_train, y_test = train_test_split(
    X_tabular, X_time_series, y, test_size=0.2, random_state=42
)

# Standardize the tabular features
scaler = StandardScaler()
X_tabular_train = scaler.fit_transform(X_tabular_train)
X_tabular_test = scaler.transform(X_tabular_test)

# Convert to PyTorch tensors
X_tabular_train_tensor = torch.FloatTensor(X_tabular_train)
X_tabular_test_tensor = torch.FloatTensor(X_tabular_test)
X_time_series_train_tensor = torch.FloatTensor(X_time_series_train).view(-1, 1, X_time_series_train.shape[1])
X_time_series_test_tensor = torch.FloatTensor(X_time_series_test).view(-1, 1, X_time_series_test.shape[1])
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

# Define the hybrid model
class HybridModel(nn.Module):
    def __init__(self, input_size_tabular, input_size_time_series, hidden_size, num_layers):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size_tabular, 64)
        self.fc2 = nn.Linear(64, 32)
        self.lstm = nn.LSTM(input_size_time_series, hidden_size, num_layers, batch_first=True)
        self.fc3 = nn.Linear(hidden_size + 32, 1)  # Combine outputs

    def forward(self, x_tabular, x_time_series):
        # Process tabular data
        tabular_out = torch.relu(self.fc1(x_tabular))
        tabular_out = torch.relu(self.fc2(tabular_out))

        # Process time series data
        lstm_out, _ = self.lstm(x_time_series)
        lstm_out = lstm_out[:, -1, :]  # Get the last time step's output

        # Combine outputs
        combined = torch.cat((tabular_out, lstm_out), dim=1)
        return self.fc3(combined)

# Initialize model, loss function, and optimizer
input_size_tabular = X_tabular_train.shape[1]
input_size_time_series = X_time_series_train.shape[2]
hidden_size = 64
num_layers = 2
model = HybridModel(input_size_tabular, input_size_time_series, hidden_size, num_layers)
criterion = nn.MSELoss()  # For regression tasks
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tabular_train_tensor, X_time_series_train_tensor)
    
    # Compute loss
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_tabular_test_tensor, X_time_series_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

In [ ]:
# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load your image dataset
train_dataset = datasets.ImageFolder(root='path_to_train_data', transform=transform)
test_dataset = datasets.ImageFolder(root='path_to_test_data', transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model, loss function, and optimizer
model = CNN(num_classes=len(train_dataset.classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')
